# Problem: Every website requires a registration for basic tasks
# Solution: Create a pseudorandom password generator

In [50]:
import string, random
def generate_password(length):
    password = ""
    for i in range(length):
        choices = string.ascii_letters + string.digits + string.punctuation
        password += random.choice(choices)
    return password

# Mongodb Local Logging

In [49]:
from pymongo import MongoClient
def log_password(username, password, url):
    client = MongoClient("localhost", 27017)
    random_db = client.random_passwords
    random_col = random_db["rng_passwords"]
    random_col.insert_one({"Username": username, 
                           "Password": password,
                           "Url":      url})

# Test this for stuff.co.nz

In [63]:
password = generate_password(10)
print("Randomly generated password is:", password)
log_password("johndoe@gmail.com", password, "https://my.stuff.co.nz")

Randomly generated password is: GDxp-|8yfX


# Create function to retrieve a record

In [67]:
def retrieve(username, url=None):
    client = MongoClient("localhost", 27017)
    random_db = client.random_passwords
    random_col = random_db["rng_passwords"]
    if url is not None:
        return random_col.find_one({"Username": username, "Url": url})
    return random_col.find({"Username": username})

# Test this function is correct

In [71]:
def test_password_entry(username, password, url):
    user_record = retrieve(username, url)
    if user_record is None:
        return False
    return user_record["Password"] == password

test_password_entry("johndoe@gmail.com", password, "https://my.stuff.co.nz")

True

# Send generated password to the clipboard

In [66]:
import pyperclip
pyperclip.copy(generate_password(10))
print(pyperclip.paste())

')`DA?y&ub


# Bundle all functionality together

In [73]:
import pyperclip
def password_package(username, url, length=10):
    password = generate_password(length)
    log_password(username, password, url)
    if not test_password_entry(username, password, url):
        raise Exception("Logging has failed, check db connection.")
    pyperclip.copy(password)
    print("Password has been generated, logged, and sent to clipboard.")

# Test this package

In [74]:
password_package("big_boy@gmail.com", "reddit.com")

Password has been generated, logged, and sent to clipboard.


In [75]:
retrieve(username="big_boy@gmail.com", url="reddit.com")

{'Password': '>cKoKD!|hG',
 'Url': 'reddit.com',
 'Username': 'big_boy@gmail.com',
 '_id': ObjectId('5997dff6c3372d230cc98326')}

In [76]:
test_password_entry(password=">cKoKD!|hG", url="reddit.com", 
                    username="big_boy@gmail.com")

True